# CNN Text Classification

- reference
    - https://ratsgo.github.io/natural%20language%20processing/2017/03/19/CNN/
    - https://github.com/bhaveshoswal/CNN-text-classification-keras
    - https://cloud.google.com/blog/big-data/2017/10/intro-to-text-classification-with-keras-automatically-tagging-stack-overflow-posts

tensorflow tokenizer 사용

In [ ]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import random


def make_input(documents):
    max_document_length = 100
    # tensorflow.contrib.learn.preprocessing 내에 VocabularyProcessor라는 클래스를 이용
    # 모든 문서에 등장하는 단어들에 인덱스를 할당
    # 길이가 다른 문서를 max_document_length로 맞춰주는 역할
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document_length)  # 객체 선언
    x = np.array(list(vocab_processor.fit_transform(documents)))
    ### 텐서플로우 vocabulary processor
    # Extract word:id mapping from the object.
    # word to ix 와 유사
    vocab_dict = vocab_processor.vocabulary_._mapping
    # Sort the vocabulary dictionary on the basis of values(id).
    sorted_vocab = sorted(vocab_dict.items(), key=lambda x: x[1])
    # Treat the id's as index into list and create a list of words in the ascending order of id's
    # word with id i goes at index i of the list.
    vocabulary = list(list(zip(*sorted_vocab))[0])
    return x, vocabulary, len(vocab_processor.vocabulary_)

def make_output(category):
    classes = pd.read_csv('./paper_class', header=None,)
    one_hot_vectors = np.eye(len(classes), dtype=int)
    class_vectors = {}
    y = []
    for i, cls in enumerate(list(classes[0])):
        class_vectors[cls] = one_hot_vectors[i]
    for c in category:
        y.append(class_vectors[c])
    return np.array(y)


def load_data():
    cnn = pd.read_excel('./paper_subject_nlp.xlsx')
    contents = cnn['subject_nlp']
    category = cnn['paper_category']

    x, vocabulary, vocabulary_size = make_input(contents)

    y = make_output(category)
    return x, y, vocabulary, vocabulary_size


In [ ]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split

print('Loading data')

x, y, vocabulary, vocabulary_size = load_data()
print("x shape {}".format(x.shape))
print("vocabulary_size {}".format(vocabulary_size))
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.05, random_state=42)

In [ ]:
vocabulary_dict = {k: v for v, k in enumerate(vocabulary)}
# Save
np.save('vocabulary_dict.npy', vocabulary_dict) 

In [ ]:
sequence_length = x.shape[1]
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 20
batch_size = 30

# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=10, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
print("Traning Model...")
early_stopping = EarlyStopping()
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint, early_stopping], validation_data=(X_test, y_test))  # starts training

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])